# A. State preparation with Jaynes-Cummings controls

In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

As a preliminary step, we consider state preparation
of a target state starting from a pure state. In addition,
we assume that any coupling to an external environment
is negligible and that the parametrized controls can be
implemented perfectly.

Here no feedback is required, we are just testing the parameterized gates setup.

As a first example, we consider the state preparation
of a cavity resonantly coupled to an externally driven
qubit

Here, we consider a particular sequence of
parametrized unitary gates originally introduced by Law
and Eberly

In [2]:
N_cav = 30

In [3]:
def qubit_unitary(alpha_re, alpha_im):
    alpha = alpha_re + 1j * alpha_im
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [4]:
def qubit_cavity_unitary(beta_re):
    beta = beta_re
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

In [5]:
# Uq = qubit_unitary(0.1, 0.1)
# Uqc = qubit_cavity_unitary(0.2, 0.2)
# print(
#     "Uq unitary check:",
#     jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
# )
# print(
#     "Uqc unitary check:",
#     jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
# )

In [6]:
# qubit_unitary(0.1, 0.1).shape

In [7]:
# qubit_unitary(0.1, 0.1)

In their groundbreaking work, Law and Eberly have
shown that any arbitrary superposition of Fock states with
maximal excitation number N can be prepared out of the
ground state in a sequence of N such interleaved gates,
also providing an algorithm to find the correct angles and
interaction durations

In [8]:
# print(qubit_cavity_unitary(0.1, 0.1).shape)

In [9]:
# qubit_cavity_unitary(0.1, 0.1)

## First target is the state $ | 1 ⟩ + | 3 ⟩ $ 

In [10]:
# TODO: Why if time steps are less than 5 then it plateus at 0.5
time_steps = 5  # corressponds to maximal excitation number of an arbitrary Fock State Superposition

In [11]:
psi0 = tensor(basis(N_cav), basis(2))
psi0 = psi0 / jnp.linalg.norm(psi0)
psi_target = tensor((fock(N_cav, 1) + fock(N_cav, 3)) / jnp.sqrt(2), basis(2))
psi_target = psi_target / jnp.linalg.norm(psi_target)

In [12]:
psi0.shape

(60, 1)

In [13]:
psi_target.shape

(60, 1)

In [14]:
print(fock(N_cav, 1))

[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


Law and Eberly provided an algorithm to determine the correct parameters for state preparation. These include:

- The rotation angle $ |\alpha| $,
- The azimuthal angle $ \arg\left(\frac{\alpha}{|\alpha|}\right) $,
- The interaction duration $ |\beta| $. <br>

So Goal is to find the best control vector (rather than control amplitudes, this time) that leads to finding the optimal state-preparation strategies. Performing as well as the Law-Eberly algorithm.

In [15]:
# Answer: see why if initial [paramaters are complex, adam fidelity is so bad and l-bfgs does error]
# --> they only train real parameters, so we need to split the complex parameters into real and imaginary parts
import jax.random as random

key = random.PRNGKey(42)  # Random seed for reproducibility
num_gates = len(
    [qubit_unitary, qubit_cavity_unitary]
)  # Number of parameterized gates
# doing a .tolist() will lead to weird shape if you multiply outside with 5 or sth
# playing wiht the range will increase or decrease fidelity
initial_parameters = [[[1.0,2.1],[0.1]] for _ in range(time_steps)]  # Initial parameters for each gate

print(f"Initial parameters: \n {initial_parameters}")

Initial parameters: 
 [[[1.0, 2.1], [0.1]], [[1.0, 2.1], [0.1]], [[1.0, 2.1], [0.1]], [[1.0, 2.1], [0.1]], [[1.0, 2.1], [0.1]]]


## Optimizing
Currently l-bfgs with the same learning rate of 0.3 converges at a local minimum of 0.5, adam also converges at 0.5 but at smaller learning rates

In [ ]:
from feedback_grape.utils.fidelity import ket2dm
import jax
key = jax.random.PRNGKey(42)

qub_unitary = {
    "gate": qubit_unitary,
    "initial_params": jax.random.uniform(
        key,
        shape=(1, 2),  # 2 for gamma and delta
        minval=-jnp.pi,
        maxval=jnp.pi,
    )[0].tolist(),
    "measurement_flag": False,
    "param_constrains": [[0, 0.5], [-1, 1]],
}

qub_cav = {
    "gate": qubit_cavity_unitary,
    "initial_params": jax.random.uniform(
        key,
        shape=(1, 1),  # 2 for gamma and delta
        minval=-jnp.pi,
        maxval=jnp.pi,
    )[0].tolist(),
    "measurement_flag": False,
    "param_constrains": [[0, 0.5]],
}

system_params = [qub_unitary, qub_cav]


result = optimize_pulse_with_feedback(
    U_0=ket2dm(psi0),
    C_target=ket2dm(psi_target),
    system_params=system_params,
    num_time_steps=time_steps,
    max_iter=1000,
    convergence_threshold=1e-16,
    type="density",
    mode="no-measurement",
    # decay={
    #     "decay_indices": [0],  # indices of gates before which decay occurs
    #     # c_ops need to be tensored with the identity operator for the cavity
    #     # because it is used directly in the lindblad equation
    #     "c_ops": {
    #         # weird behavior when gamma is 0.00
    #         "tm": [tensor(identity(N_cav), jnp.sqrt(0.15) * sigmam())],
    #         # "tc": [tensor(identity(N_cav), jnp.sqrt(0.15) * sigmap())],
    #     },  # c_ops for each decay index
    #     "tsave": jnp.linspace(0, 1, 2),  # time grid for decay
    #     "Hamiltonian": None,
    # },
    goal="fidelity",
    learning_rate=0.02,
    batch_size=10,
    eval_batch_size=2,
)

ValueError: If you provide parameter constraints for some gates, you need to provide them for all gates.

In [ ]:
len(result.returned_params)

5

In [ ]:
result.returned_params

[[Array([[ 0.90861841, -6.21714015],
         [ 0.90861841, -6.21714015]], dtype=float64),
  Array([[-3.56962488],
         [-3.56962488]], dtype=float64)],
 [Array([[-1.59090615e-06,  3.14159262e+00],
         [-1.59090615e-06,  3.14159262e+00]], dtype=float64),
  Array([[2.77362683],
         [2.77362683]], dtype=float64)],
 [Array([[3.14159265e+00, 6.01376749e-07],
         [3.14159265e+00, 6.01376749e-07]], dtype=float64),
  Array([[-1.48828577],
         [-1.48828577]], dtype=float64)],
 [Array([[1.48767291, 6.10452689],
         [1.48767291, 6.10452689]], dtype=float64),
  Array([[2.52600011],
         [2.52600011]], dtype=float64)],
 [Array([[ 2.48162667e-07, -3.14159279e+00],
         [ 2.48162667e-07, -3.14159279e+00]], dtype=float64),
  Array([[3.14159256],
         [3.14159256]], dtype=float64)]]

In [ ]:
print(result.final_state)

[[[ 1.38779799e-16-4.46897775e-32j -3.85622920e-16+1.36123412e-15j
   -4.22559145e-09+7.17873750e-09j ...  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
  [-3.85622918e-16-1.36123412e-15j  1.44132417e-14-2.92357670e-32j
    8.21548050e-08+2.14997683e-08j ...  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
  [-4.22559145e-09-7.17873750e-09j  8.21548050e-08-2.14997683e-08j
    4.99999969e-01+1.32348898e-23j ...  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
  ...
  [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
  [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
    0.00000000e+00+0.00000000e+0

In [ ]:
print(result)

FgResult(optimized_trainable_parameters=[[Array([ 0.90861841, -6.21714015], dtype=float64), Array([-3.56962488], dtype=float64)], [Array([-1.59090615e-06,  3.14159262e+00], dtype=float64), Array([2.77362683], dtype=float64)], [Array([3.14159265e+00, 6.01376749e-07], dtype=float64), Array([-1.48828577], dtype=float64)], [Array([1.48767291, 6.10452689], dtype=float64), Array([2.52600011], dtype=float64)], [Array([ 2.48162667e-07, -3.14159279e+00], dtype=float64), Array([3.14159256], dtype=float64)]], iterations=627, final_state=Array([[[ 1.38779799e-16-4.46897775e-32j,
         -3.85622920e-16+1.36123412e-15j,
         -4.22559145e-09+7.17873750e-09j, ...,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j],
        [-3.85622918e-16-1.36123412e-15j,
          1.44132417e-14-2.92357670e-32j,
          8.21548050e-08+2.14997683e-08j, ...,
          0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+

In [ ]:
result.final_fidelity

Array(1., dtype=float64)

In [ ]:
from feedback_grape.fgrape_helpers import prepare_parameters_from_dict
def get_trainable_parameters(initial_parameters, param_constrains, num_time_steps, rng_key):
    trainable_params = []
    flat_params ,_ = prepare_parameters_from_dict(initial_parameters)
    trainable_params.append(flat_params)
    for i in range(num_time_steps - 1):
        gate_params_list = []
        if param_constrains != []:
            for gate_params, gate_constraints in zip(flat_params, param_constrains):
                sampled_params = []
                for var_bounds in gate_constraints:
                    rng_key, subkey = jax.random.split(rng_key)
                    var = jax.random.uniform(
                        subkey,
                        shape=(),
                        minval=var_bounds[0],
                        maxval=var_bounds[1],
                    )
                    sampled_params.append(var)
                gate_params_list.append(jnp.array(sampled_params))
        else:
            for gate_params in flat_params:
                sampled_params = []
                for _ in range(gate_params.shape[0]):
                    rng_key, subkey = jax.random.split(rng_key)
                    var = jax.random.uniform(
                        subkey,
                        shape=(),
                        minval=-jnp.pi,
                        maxval=jnp.pi,
                    )
                    sampled_params.append(var)
                gate_params_list.append(jnp.array(sampled_params))
        trainable_params.append(gate_params_list)

    return trainable_params


In [ ]:
initial_params = {
    "qubit_unitary": [0.1, 0.1],
    "qubit_cavity_unitary": [0.2, 0.2],
}
# param_constrains = [[[0, 0.5], [0.5, 1.0]], [[1.0, 1.5], [1.5, 2.0]]]
param_constrains = []
num_time_steps = 3
key_1 = random.PRNGKey(42)


trainable_params = get_trainable_parameters(initial_params, param_constrains, num_time_steps, key_1)


In [ ]:
from pprint import pprint
pprint(trainable_params)

[[Array([0.1, 0.1], dtype=float64), Array([0.2, 0.2], dtype=float64)],
 [Array([1.52127861, 0.99764266], dtype=float64),
  Array([1.17781588, 2.76860586], dtype=float64)],
 [Array([-0.05248534,  0.79283613], dtype=float64),
  Array([-1.34523035,  2.7576985 ], dtype=float64)]]
